# Data exploration

## Imports

In [1]:
import os
import pandas as pd

## Simplifying the `status.csv` data

In [ ]:
status_file = lambda f: os.path.join("data/status", f)
status_blocks = sorted(os.listdir("data/status"))

In [ ]:
status_0 = pd.read_csv(status_file(status_blocks[0]), header=0)

In [ ]:
status_0["date"] = status_0["time"].map(lambda v: v.split(" ")[0])

In [ ]:
status_0.drop("time", axis=1).groupby(["station_id", "date"]).agg(["sum", "size"]).reset_index()

In [ ]:
from itertools import pairwise

columns = status_0.columns

In [ ]:
status = pd.DataFrame([], columns=["station_id", "date", "bikes_available_sum", "bikes_available_size", "docks_available_sum", "docks_available_size"])
n_blocks_per_read = 100

parts = []
for span_begin, span_end in pairwise(range(0, len(status_blocks) + n_blocks_per_read, n_blocks_per_read)):
    print(f"Concatenating blocks {span_begin:4}-{span_end:4} | Total rows: {status.shape[0]:>6}", end="\r")
    tmp = pd.concat(
        [
            # The first file "xaa" has a header, we can just skip it since we already know it
            pd.read_csv(status_file(f), names=columns, skiprows=1 if f == "xaa" else 0)
            for f in status_blocks[span_begin:span_end]
        ],
        axis=0,
        ignore_index=True
    )

    tmp["date"] = tmp["time"].map(lambda v: v.split(" ")[0])
    tmp = (tmp
        .drop("time", axis=1)
        .groupby(["station_id", "date"])
        # We have to aggregate each part with the sum
        # (and number of values of that sum) because each block
        # may not have all the data for each day-station pair,
        # so we can't just do the mean yet
        .agg(["sum", "size"])
        .reset_index()
    )
    tmp.columns = tmp.columns.to_flat_index()
    tmp.rename(inplace=True, columns={
        ("station_id", ""): "station_id",
        ("date", ""): "date",
        ("bikes_available", "sum"): "bikes_available_sum",
        ("bikes_available", "size"): "bikes_available_size",
        ("docks_available", "sum"): "docks_available_sum",
        ("docks_available", "size"): "docks_available_size",
    })

    status = pd.concat([status, tmp], axis=0, ignore_index=True)

In [ ]:
status["bikes_available_avg"] = status["bikes_available_sum"] / status["bikes_available_size"]
status["docks_available_avg"] = status["docks_available_sum"] / status["docks_available_size"]
status.drop(inplace=True, columns=["bikes_available_sum", "bikes_available_size", "docks_available_sum", "docks_available_size"])

In [ ]:
status.to_csv("data/status_small.csv", index=False)

## Simplifying the `trips.csv`

In [ ]:
trip = pd.read_csv("data/trip.csv")

In [5]:
trip

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4576,63,8/29/2013 14:13,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127
1,4607,70,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138
2,4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214
3,4251,77,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060
4,4299,83,8/29/2013 12:02,South Van Ness at Market,66,8/29/2013 12:04,Market at 10th,67,319,Subscriber,94103
...,...,...,...,...,...,...,...,...,...,...,...
669954,432951,619,9/1/2014 4:21,Powell Street BART,39,9/1/2014 4:32,Townsend at 7th,65,335,Subscriber,94118
669955,432950,6712,9/1/2014 3:16,Harry Bridges Plaza (Ferry Building),50,9/1/2014 5:08,San Francisco Caltrain (Townsend at 4th),70,259,Customer,44100
669956,432949,538,9/1/2014 0:05,South Van Ness at Market,66,9/1/2014 0:14,5th at Howard,57,466,Customer,32
669957,432948,568,9/1/2014 0:05,South Van Ness at Market,66,9/1/2014 0:15,5th at Howard,57,461,Customer,32


In [48]:
trip_simple = trip.drop(["zip_code", "id", "start_station_name", "end_station_name", "bike_id"], axis=1)

In [12]:
import datetime

In [49]:
trip_simple["start_date"] = trip_simple["start_date"].map(lambda s: datetime.datetime.strptime(s, "%m/%d/%Y %H:%M").date())
trip_simple["end_date"] = trip_simple["end_date"].map(lambda s: datetime.datetime.strptime(s, "%m/%d/%Y %H:%M").date())

In [58]:
trip_simple_final = (trip_simple
        .groupby(by=["start_station_id", "end_station_id", "start_date", "end_date", "subscription_type"])
        .agg(["mean", "size"])
        .reset_index()
)
trip_simple_final

start_station_id end_station_id  start_date    end_date   
                                                                 
0                     2              2  2013-08-30  2013-08-30  \
1                     2              2  2013-09-02  2013-09-02   
2                     2              2  2013-09-03  2013-09-03   
3                     2              2  2013-09-05  2013-09-05   
4                     2              2  2013-09-06  2013-09-06   
...                 ...            ...         ...         ...   
362483               84             84  2015-06-21  2015-06-21   
362484               84             84  2015-06-22  2015-06-22   
362485               84             84  2015-07-03  2015-07-03   
362486               84             84  2015-07-05  2015-07-05   
362487               84             84  2015-08-08  2015-08-08   

       subscription_type duration       
                             mean size  
0             Subscriber    289.0    1  
1               Customer   3626.0    2  
2               Customer   3471.0    1  
3               Customer    513.0    1  
4             Subscriber    119.5    2  
...                  ...      ...  ...  
362483          Customer   1506.2    5  
362484        Subscriber    855.0    1  
362485        Subscriber   1038.0    1  
362486          Customer   3450.0    1  
362487          Customer   3153.0    1  

[362488 rows x 7 columns]

In [59]:
trip_simple_final.columns = trip_simple_final.columns.to_flat_index()

In [62]:
trip_simple_final.rename(columns={
    ("start_station_id", ""): "start_station_id",
    ("end_station_id", ""): "end_station_id",
    ("start_date", ""): "start_date",
    ("end_date", ""): "end_date",
    ("subscription_type", ""): "subscription_type",
    ("duration", "mean"): "duration_avg",
    ("duration", "size"): "total_trips",
}).to_csv("data/trip_small.csv", index=False)

## Obtaining metadata

In [115]:
status_small = pd.read_csv("data/status_small.csv")
trip_small = pd.read_csv("data/trip_small.csv")
station = pd.read_csv("data/station.csv")

In [80]:
from datetime import datetime

In [116]:
station["installation_date"] = station["installation_date"].map(lambda s: datetime.strptime(s, "%m/%d/%Y").date().isoformat())

In [104]:
station

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,2013-08-06
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,2013-08-05
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,2013-08-06
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,2013-08-05
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,2013-08-07
...,...,...,...,...,...,...,...
65,77,Market at Sansome,37.789625,-122.400811,27,San Francisco,2013-08-25
66,80,Santa Clara County Civic Center,37.352601,-121.905733,15,San Jose,2013-12-31
67,82,Broadway St at Battery St,37.798541,-122.400862,15,San Francisco,2014-01-22
68,83,Mezes Park,37.491269,-122.236234,15,Redwood City,2014-02-20


In [117]:
stations = {s["id"]: {k: s[k] for k in s.keys() if k != "id"} for _, s in station.iterrows()}

In [122]:
# Yes these are strings, but they are ordered in a way that allows these ordinal comparisons to make sense (YYYY-mm-dd)
max_date = max(
    status_small["date"].max(),
    trip_small["start_date"].max(),
    trip_small["end_date"].max(),
    station["installation_date"].max(),
)

min_date = min(
    status_small["date"].min(),
    trip_small["start_date"].min(),
    trip_small["end_date"].min(),
    station["installation_date"].min(),
)

In [126]:
metadata = {
    "date_min": min_date,
    "date_max": max_date,
    "stations": stations,
}

In [125]:
import json

In [131]:
with open("data/metadata.json", "wt") as f:
    f.write(json.dumps(metadata, indent=4))

## Reading the other data

In [2]:
station = pd.read_csv("data/station.csv")
weather = pd.read_csv("data/weather.csv")
trip = pd.read_csv("data/trip.csv")

In [3]:
weather["zip_code"].drop_duplicates().values

array([94107, 94063, 94301, 94041, 95113])

In [ ]:
city_zipcode_map = {
    94107: "San Francisco",
    94063: "Redwood City",
    94301: "Palo Alto",
    94041: "Mountain View",
    95113: "San Jose",
}

In [5]:
trip["zip_code"].drop_duplicates().count()

7439

In [6]:
station.shape

(70, 7)

In [7]:
station["city"].drop_duplicates().values

array(['San Jose', 'Redwood City', 'Mountain View', 'Palo Alto',
       'San Francisco'], dtype=object)

In [6]:
weather["events"].drop_duplicates().values

array([nan, 'Fog', 'Rain', 'Fog-Rain', 'rain', 'Rain-Thunderstorm'],
      dtype=object)

## Testing the small data

In [2]:
trip_small = pd.read_csv("data/trip_small.csv")

In [7]:
import datetime

In [9]:
trip_small["start_date"] = trip_small["start_date"].map(datetime.date.fromisoformat)
trip_small["end_date"] = trip_small["end_date"].map(datetime.date.fromisoformat)

In [12]:
trip_small["diff"] = trip_small["end_date"] - trip_small["start_date"]

In [19]:
trip_small[["diff", "total_trips"]].groupby("diff").sum()

,total_trips
diff,
0 days,667860
1 days,1915
2 days,91
3 days,47
4 days,15
5 days,6
6 days,8
7 days,8
8 days,5


# Diogo's fuckAroundAndFindOut

### Creation of bikes_usage

In [8]:
trips = pd.read_csv("trip.csv")
trips_small = trips [["duration","bike_id","start_date"]]

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013
...,...,...,...,...,...,...,...
65,77,Market at Sansome,37.789625,-122.400811,27,San Francisco,8/25/2013
66,80,Santa Clara County Civic Center,37.352601,-121.905733,15,San Jose,12/31/2013
67,82,Broadway St at Battery St,37.798541,-122.400862,15,San Francisco,1/22/2014
68,83,Mezes Park,37.491269,-122.236234,15,Redwood City,2/20/2014


In [ ]:
import datetime
trips_small["start_date"] = trips_small["start_date"].map(lambda s: datetime.datetime.strptime(s, "%m/%d/%Y %H:%M").date())

In [ ]:
# Group by 'start_date' and 'bike_id', then sum the durations
total_duration_per_day = trips_small.groupby(['start_date', 'bike_id'])['duration'].sum().reset_index(name='total_duration')

total_duration_per_day

In [ ]:
#save it
total_duration_per_day.to_csv("bikes_usage.csv",index=False)